# PyCity Schools

Overall, the district has almost 40K students with a budget of approximately $24.5 million.  On average, students receive a C grade in math and reading.
-  Bucketing the schools by spend we see those that spend more per student do not outperform those that spend less per student.
-  When bucketing the schools by size we see the smaller schools outperforming the larger schools in math and reading while having overall higher pass rates.
-  When segmenting based on school type (charter vs district), the charter schools show significantly higher outcomes in terms of math and reading scores.
-  At the least we can conclude that a relatively large budget is not a cure-all for academic performance.
-  In order to determine causal factors leading to better performance of charter schools additional analysis would be required.
-  Further analyis might include:
>-  Data on family income per student and parent education per student.  These could help control for variables outside a school's influence.
>-  Data on average class size or teacher/student ratio per school.  It's unclear why the size (total number of students) per school would affect performance.
>-  To determine if this is related more to class size it would be helpful to have a teacher/student ratio per school.
>-  Finally, it would be helpful to seek out data from additional school districts, if possible.  One could then have a larger data set to explore potential correlations.
>-  Specifically, I would look only within public schools to determine if there is a correlation between overall school size and performance.  If none is found, the 
    hypothesis that total number of students at a school lowers overall performance is weakened.

In [1]:
#Dependencies
import pandas as pd
import numpy as np
import os

In [2]:
#Source data files
school_data = os.path.join('.\Resources','schools_complete.csv')
student_data = os.path.join('.\Resources','students_complete.csv')


In [3]:
#Read in csv data
load_school = pd.read_csv(school_data)
load_student = pd.read_csv(student_data)

In [4]:
#Quick summary stats of school data set
school_describe = load_school.describe()
school_describe

,School ID,size,budget
count,15.000000,15.000000,1.500000e+01
mean,7.000000,2611.333333,1.643295e+06
std,4.472136,1420.915282,9.347763e+05
min,0.000000,427.000000,2.480870e+05
25%,3.500000,1698.000000,1.046265e+06
50%,7.000000,2283.000000,1.319574e+06
75%,10.500000,3474.000000,2.228999e+06
max,14.000000,4976.000000,3.124928e+06


In [5]:
#Total Schools
total_schools = int(school_describe.iloc[0,0])
#Total District Budget
total_budget = load_school['budget'].sum()


In [6]:
student_describe = load_student.describe()
student_describe

,Student ID,reading_score,math_score
count,39170.000000,39170.00000,39170.000000
mean,19584.500000,81.87784,78.985371
std,11307.549359,10.23958,12.309968
min,0.000000,63.00000,55.000000
25%,9792.250000,73.00000,69.000000
50%,19584.500000,82.00000,79.000000
75%,29376.750000,91.00000,89.000000
max,39169.000000,99.00000,99.000000


In [7]:
#Total number of students
total_students = int(student_describe.iloc[0,0])
#Average reading score
avg_reading = round(student_describe.iloc[1,1],1)
#Average math score
avg_math = round(student_describe.iloc[1,2],1)


In [8]:
#Percent passing reading
prcnt_pass_read = round((load_student.loc[load_student['reading_score'] >= 70,'student_name'].count()/total_students)*100,1)

#print(f"{round(prcnt_pass_read,3)*100}%")


In [9]:
#Percent passing reading
prcnt_pass_math = round((load_student.loc[load_student['math_score'] >= 70,'student_name'].count()/total_students)*100,1)

#print(f"{round(prcnt_pass_math,3)*100}%")

In [10]:
#Overall pass rate
overall = round(((prcnt_pass_read + prcnt_pass_math)/2),1)
print(f"{overall}%")

80.4%


In [11]:
#District Summary Table
district_sum_headers = ['Total Schools','Total Students', 'Total Budget', 'Average Math Score','Average Reading Score','% Passing Math', '% Passing Reading','Overall Pass Rate']
district_data = [total_schools,total_students,total_budget,avg_math,avg_reading,prcnt_pass_math,prcnt_pass_read,overall]
district_summary = pd.DataFrame({'Metric':district_sum_headers,'Result':district_data})
district_summary.set_index('Metric',inplace=True)
district_summary_tran = district_summary.T



## District Summary

In [12]:

district_summary_tran[['Total Schools','Total Students']] = district_summary_tran[['Total Schools','Total Students']].astype(int)


district_summary_tran['Overall Pass Rate'] = pd.Series(["{0:.1f}%".format(val) for val in district_summary_tran['Overall Pass Rate']], index = district_summary_tran.index)
district_summary_tran['Total Budget'] = pd.Series(["${0:.2f}".format(val) for val in district_summary_tran['Total Budget']], index = district_summary_tran.index)
district_summary_tran

Metric,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Result,15,39170,$24649428.00,79.0,81.9,75.0,85.8,80.4%


In [13]:
#Merge two data sets based on school_name column
all_data = load_student.merge(load_school,on='school_name',how='inner')
all_data.drop(['School ID','Student ID'],axis=1, inplace=True)
all_data.columns = ['Student Name','Gender','Grade','School Name','Reading Score','Math Score','Type','Size','Budget']
all_data.head()

,Student Name,Gender,Grade,School Name,Reading Score,Math Score,Type,Size,Budget
0,Paul Bradley,M,9th,Huang High School,66,79,District,2917,1910635
1,Victor Smith,M,12th,Huang High School,94,61,District,2917,1910635
2,Kevin Rodriguez,M,12th,Huang High School,90,60,District,2917,1910635
3,Dr. Richard Scott,M,12th,Huang High School,67,58,District,2917,1910635
4,Bonnie Ray,F,9th,Huang High School,97,84,District,2917,1910635


In [14]:
#Generate dataframe of mean scores
school_sum_df = all_data.groupby('School Name').mean()
school_sum_df['$ Per Student'] = school_sum_df['Budget']/school_sum_df['Size']
#school_sum_df.rename_axis('School Name',inplace=True)
#school_sum_df.index.name = 'School Name'
school_sum_df.head()

,Reading Score,Math Score,Size,Budget,$ Per Student
School Name,,,,,
Bailey High School,81.033963,77.048432,4976.0,3124928.0,628.0
Cabrera High School,83.975780,83.061895,1858.0,1081356.0,582.0
Figueroa High School,81.158020,76.711767,2949.0,1884411.0,639.0
Ford High School,80.746258,77.102592,2739.0,1763916.0,644.0
Griffin High School,83.816757,83.351499,1468.0,917500.0,625.0


In [15]:
#Get percent passing reading and math

pass_df = all_data.groupby('School Name')[['Reading Score','Math Score']].apply(lambda x: (x >= 70).sum()).reset_index()
#df11=df.groupby('key1')['key2'].apply(lambda x: (x=='one').sum()).reset_index(name='count')
pass_df = pass_df.set_index(['School Name'])
pass_df['% Passing Math'] = round((pass_df['Math Score']/school_sum_df['Size'])*100,1)
pass_df['% Passing Reading'] = round((pass_df['Reading Score']/school_sum_df['Size'])*100,1)   
pass_df

,Reading Score,Math Score,% Passing Math,% Passing Reading
School Name,,,,
Bailey High School,4077,3318,66.7,81.9
Cabrera High School,1803,1749,94.1,97.0
Figueroa High School,2381,1946,66.0,80.7
Ford High School,2172,1871,68.3,79.3
Griffin High School,1426,1371,93.4,97.1
Hernandez High School,3748,3094,66.8,80.9
Holden High School,411,395,92.5,96.3
Huang High School,2372,1916,65.7,81.3
Johnson High School,3867,3145,66.1,81.2


In [16]:
#Select columns and create new df with only % passing columns
sub_prcnt_df = pass_df[['% Passing Math','% Passing Reading']]
#Add % pass columns to schools summary
final_school_df = pd.concat([school_sum_df,sub_prcnt_df],axis=1,sort=True)


## Summary by School

In [17]:
#Add overall passing based on % math and % read passing
final_school_df['Overall Passing'] = (final_school_df['% Passing Math'] + final_school_df['% Passing Reading'])/2
final_school_show = final_school_df.copy(deep=True)
final_school_show['Budget'] = pd.Series(["$ {0:.2f}".format(val) for val in final_school_df['Budget']], index = final_school_df.index)
final_school_show['$ Per Student'] = pd.Series(["$ {0:.2f}".format(val) for val in final_school_df['$ Per Student']], index = final_school_df.index)
final_school_show['Overall Passing'] = pd.Series(["{0:.1f}%".format(val) for val in final_school_df['Overall Passing']], index = final_school_df.index)
final_school_show[['Reading Score','Math Score']] = final_school_df[['Reading Score','Math Score']].round(2)
final_school_show['Size'] = final_school_df['Size'].astype(int)
final_school_show

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
School Name,,,,,,,,
Bailey High School,81.03,77.05,4976,$ 3124928.00,$ 628.00,66.7,81.9,74.3%
Cabrera High School,83.98,83.06,1858,$ 1081356.00,$ 582.00,94.1,97.0,95.5%
Figueroa High School,81.16,76.71,2949,$ 1884411.00,$ 639.00,66.0,80.7,73.3%
Ford High School,80.75,77.10,2739,$ 1763916.00,$ 644.00,68.3,79.3,73.8%
Griffin High School,83.82,83.35,1468,$ 917500.00,$ 625.00,93.4,97.1,95.2%
Hernandez High School,80.93,77.29,4635,$ 3022020.00,$ 652.00,66.8,80.9,73.8%
Holden High School,83.81,83.80,427,$ 248087.00,$ 581.00,92.5,96.3,94.4%
Huang High School,81.18,76.63,2917,$ 1910635.00,$ 655.00,65.7,81.3,73.5%
Johnson High School,80.97,77.07,4761,$ 3094650.00,$ 650.00,66.1,81.2,73.7%


## Top Five Schools by Overall Passing Percentage

In [18]:
#Show top five schools sorted by overall passing percentage
top_five = final_school_show.sort_values('Overall Passing',ascending=False).head()
top_five

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
School Name,,,,,,,,
Cabrera High School,83.98,83.06,1858,$ 1081356.00,$ 582.00,94.1,97.0,95.5%
Thomas High School,83.85,83.42,1635,$ 1043130.00,$ 638.00,93.3,97.3,95.3%
Griffin High School,83.82,83.35,1468,$ 917500.00,$ 625.00,93.4,97.1,95.2%
Pena High School,84.04,83.84,962,$ 585858.00,$ 609.00,94.6,95.9,95.2%
Wilson High School,83.99,83.27,2283,$ 1319574.00,$ 578.00,93.9,96.5,95.2%


## Bottom Five Schools by Overall Passing Percentage

In [19]:
#Show bottom five schools sorted by overall passing percentage
bottom_five = final_school_show.sort_values('Overall Passing',ascending=False).tail()
bottom_five

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
School Name,,,,,,,,
Hernandez High School,80.93,77.29,4635,$ 3022020.00,$ 652.00,66.8,80.9,73.8%
Johnson High School,80.97,77.07,4761,$ 3094650.00,$ 650.00,66.1,81.2,73.7%
Huang High School,81.18,76.63,2917,$ 1910635.00,$ 655.00,65.7,81.3,73.5%
Figueroa High School,81.16,76.71,2949,$ 1884411.00,$ 639.00,66.0,80.7,73.3%
Rodriguez High School,80.74,76.84,3999,$ 2547363.00,$ 637.00,66.4,80.2,73.3%


## Math and Reading Averages by Grade and School

In [20]:
#Table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
#Replace the strings for grade level with integers so they can be sorted automatically
mean_by_grade = all_data.replace(['9th','10th','11th','12th'],[9,10,11,12])
#Group by school and grade and calculate mean math and reading score
mean_grade_summary = pd.pivot_table(mean_by_grade, values=['Reading Score','Math Score'], index=['School Name','Grade'], aggfunc=np.mean)
mean_grade_summary[['Math Score', 'Reading Score']]=mean_grade_summary[['Math Score', 'Reading Score']].round(2)
mean_grade_summary.rename(columns={'Math Score': 'Avg Math Score','Reading Score':'Avg Reading Score'},inplace=True)
#mean_grade_summary['Math Score','Reading Score'].astype('int')
mean_grade_summary



Avg Math Score  Avg Reading Score
School Name           Grade                                   
Bailey High School    9               77.08              81.30
                      10              77.00              80.91
                      11              77.52              80.95
                      12              76.49              80.91
Cabrera High School   9               83.09              83.68
                      10              83.15              84.25
                      11              82.77              83.79
                      12              83.28              84.29
Figueroa High School  9               76.40              81.20
                      10              76.54              81.41
                      11              76.88              80.64
                      12              77.15              81.38
Ford High School      9               77.36              80.63
                      10              77.67              81.26
                      11              76.92              80.40
                      12              76.18              80.66
Griffin High School   9               82.04              83.37
                      10              84.23              83.71
                      11              83.84              84.29
                      12              83.36              84.01
Hernandez High School 9               77.44              80.87
                      10              77.34              80.66
                      11              77.14              81.40
                      12              77.19              80.86
Holden High School    9               83.79              83.68
                      10              83.43              83.32
                      11              85.00              83.82
                      12              82.86              84.70
Huang High School     9               77.03              81.29
                      10              75.91              81.51
                      11              76.45              81.42
                      12              77.23              80.31
Johnson High School   9               77.19              81.26
                      10              76.69              80.77
                      11              77.49              80.62
                      12              76.86              81.23
Pena High School      9               83.63              83.81
                      10              83.37              83.61
                      11              84.33              84.34
                      12              84.12              84.59
Rodriguez High School 9               76.86              80.99
                      10              76.61              80.63
                      11              76.40              80.86
                      12              77.69              80.38
Shelton High School   9               83.42              84.12
                      10              82.92              83.44
                      11              83.38              84.37
                      12              83.78              82.78
Thomas High School    9               83.59              83.73
                      10              83.09              84.25
                      11              83.50              83.59
                      12              83.50              83.83
Wilson High School    9               83.09              83.94
                      10              83.72              84.02
                      11              83.20              83.76
                      12              83.04              84.32
Wright High School    9               83.26              83.83
                      10              84.01              83.81
                      11              83.84              84.16
                      12              83.64              84.07

In [21]:
#Summary statistics to help with deriving bins
final_school_df['$ Per Student'].describe()

count     15.000000
mean     620.066667
std       28.544368
min      578.000000
25%      591.500000
50%      628.000000
75%      641.500000
max      655.000000
Name: $ Per Student, dtype: float64

In [22]:
final_school_df.dtypes

Reading Score        float64
Math Score           float64
Size                 float64
Budget               float64
$ Per Student        float64
% Passing Math       float64
% Passing Reading    float64
Overall Passing      float64
dtype: object

## Summary by Spend Buckets

In [23]:
bins = [0,598,618,638,658]
spend_labels = ['Low','Low-Mid','Mid','High']
#df["Test Score Summary"] = pd.cut(df["Test Score"], bins, labels=group_names)
final_school_df['Spend Category'] = pd.cut(final_school_df['$ Per Student'],bins,labels=spend_labels)
by_spend_df = final_school_df.groupby(['Spend Category']).mean()

by_spend_df['Budget'] = pd.Series(["$ {0:.2f}".format(val) for val in by_spend_df['Budget']], index = by_spend_df.index)
by_spend_df['$ Per Student'] = pd.Series(["$ {0:.2f}".format(val) for val in by_spend_df['$ Per Student']], index = by_spend_df.index)
by_spend_df['Overall Passing'] = pd.Series(["{0:.1f}%".format(val) for val in by_spend_df['Overall Passing']], index = by_spend_df.index)
by_spend_df[['Reading Score','Math Score','% Passing Reading','% Passing Math']] = by_spend_df[['Reading Score','Math Score','% Passing Reading','% Passing Math']].round(1)
by_spend_df['Size'] = by_spend_df['Size'].astype(int)
by_spend_df

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
Spend Category,,,,,,,,
Low,83.9,83.5,1592,$ 924604.25,$ 581.00,93.4,96.6,95.0%
Low-Mid,83.9,83.6,1361,$ 821229.00,$ 604.50,94.2,95.9,95.1%
Mid,82.4,80.2,3019,$ 1908230.25,$ 632.00,80.0,89.1,84.5%
High,81.0,77.0,3600,$ 2335126.40,$ 648.00,66.6,80.7,73.6%


In [24]:
#Get sum stats of Size to use for bins
final_school_df['Size'].describe()

count      15.000000
mean     2611.333333
std      1420.915282
min       427.000000
25%      1698.000000
50%      2283.000000
75%      3474.000000
max      4976.000000
Name: Size, dtype: float64

## Summary by Size Buckets

In [25]:
#Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).
size_bins = [0,1700,3400,5100]
size_labels = ['Small','Medium','Large']
final_school_df['Size Category'] = pd.cut(final_school_df['Size'],size_bins,labels=size_labels)
by_size_df = final_school_df.groupby(['Size Category']).mean()
by_size_df['Budget'] = pd.Series(["$ {0:.2f}".format(val) for val in by_size_df['Budget']], index = by_size_df.index)
by_size_df['$ Per Student'] = pd.Series(["$ {0:.2f}".format(val) for val in by_size_df['$ Per Student']], index = by_size_df.index)
by_size_df['Overall Passing'] = pd.Series(["{0:.1f}%".format(val) for val in by_size_df['Overall Passing']], index = by_size_df.index)
by_size_df[['Reading Score','Math Score','% Passing Reading','% Passing Math']] = by_size_df[['Reading Score','Math Score','% Passing Reading','% Passing Math']].round(1)
by_size_df['Size'] = by_size_df['Size'].astype(int)
#by_size_df.columns = final_headers
by_size_df

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
Size Category,,,,,,,,
Small,83.9,83.6,1123,$ 698643.75,$ 613.25,93.4,96.6,95.0%
Medium,82.7,80.5,2329,$ 1437984.57,$ 611.57,82.2,89.6,85.9%
Large,80.9,77.1,4592,$ 2947240.25,$ 641.75,66.5,81.0,73.8%


## Summary by School Type

In [26]:
#Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).
#Generate df of unique schools and the type
unique_schools_type = all_data.drop_duplicates(subset='School Name',keep='first').set_index('School Name').sort_values(['School Name'])['Type']
#Concat the unique df with existing school summary df
final_school_df['Type'] = unique_schools_type
#Group by type and calculate means
group_by_type = final_school_df.groupby(['Type']).mean()
by_size_df = final_school_df.groupby(['Size Category']).mean()
group_by_type['Budget'] = pd.Series(["$ {0:.2f}".format(val) for val in group_by_type['Budget']], index = group_by_type.index)
group_by_type['$ Per Student'] = pd.Series(["$ {0:.2f}".format(val) for val in group_by_type['$ Per Student']], index = group_by_type.index)
group_by_type['Overall Passing'] = pd.Series(["{0:.1f}%".format(val) for val in group_by_type['Overall Passing']], index = group_by_type.index)
group_by_type[['Reading Score','Math Score','% Passing Reading','% Passing Math']] = group_by_type[['Reading Score','Math Score','% Passing Reading','% Passing Math']].round(1)
group_by_type['Size'] = group_by_type['Size'].astype(int)

group_by_type

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
Type,,,,,,,,
Charter,83.9,83.5,1524,$ 912688.12,$ 599.50,93.6,96.6,95.1%
District,81.0,77.0,3853,$ 2478274.71,$ 643.57,66.6,80.8,73.7%
